In [1]:
import torch
from torch import nn
from torch.utils import data
import os
import numpy as np
from torch.utils.data import Sampler, BatchSampler,SequentialSampler, RandomSampler, DataLoader
from skimage import io, transform
import matplotlib.pyplot as plt
from torchvision import transforms, utils, models

In [2]:
from utils import *
from losses import *
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable
import time

In [3]:
import gc

In [4]:
def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

In [5]:
def create_filelists(data_folder):
    identities = os.listdir(data_folder)
    identities.sort()
    n_identities = len(identities)
    data_source = []
    Filelist = []
    idx = 0
    for identity in identities:
        filenames = listdir_fullpath(data_folder + identity)
        filenames.sort()
        samples = [{'filename':filenames[i],'idx':idx+i} for i in range(len(filenames))]
        data_source.append(samples)
        Filelist.extend(filenames)
        idx = idx + len(filenames)
    return data_source, Filelist, identities

In [6]:
class PKSampler(Sampler):
    def __init__(self, data_source, p , k):
        self.data_source = data_source #list of lists containing id and filenames and index in total filelist
        self.p = p
        self.k = k
        self.dataset_size = data_source[-1][-1]['idx']
        self.length = self.dataset_size // ((len(self.data_source) // self.p) * self.p )
        
    def __iter__(self):
        id_list_len = len(self.data_source)
        batch = []
        
        for n_pass in range(self.dataset_size // (((id_list_len // self.p) * self.p ) * self.k)):
            random_id_list = np.random.permutation(id_list_len)
            id_idx = 0
            for r_id in range((id_list_len//self.p)-1):
                if id_idx>=id_list_len:
                    break
                for p_i in range(self.p):
                    id_idx = id_idx + 1
                    random_sample_list = np.random.permutation(len(self.data_source[random_id_list[id_idx]]))
                    for k_i in range(self.k):
                        batch.append(self.data_source[random_id_list[id_idx]][random_sample_list[k_i]]['idx'])
                yield batch
                batch = []
                
    def __len__(self):
        return self.length

In [7]:
class VGGFace2_100_PK(data.Dataset):
    def __init__(self, Filelist, identities, cropsize=[224,224] ,transform=None):
        self.transform = transform
        self.cropsize = cropsize
        self.Filelist = Filelist
        self.identities = identities
                    
    def __len__(self):
        return len(self.Filelist)

    def __getitem__(self, index):
        img_name = self.Filelist[index]
        image = io.imread(img_name)/255.0
              
        h,w = image.shape[:2]
#         print(h,w)
        min_dim = min(h,w)
#         print(min_dim)
        
        if min_dim <= 224:
            image = transform.rescale(image, 228.0/min_dim, mode="constant")
            h,w = image.shape[:2]
#             print("resized = ",h,w)
        
        
        new_h,new_w = self.cropsize
        
#         print(h,w,new_h,new_w)
        
        top = np.random.randint(0, h - new_h )
        left = np.random.randint(0, w - new_w )
            
        image = image[ top : top + new_h,
                      left : left + new_w ]
        
#         plt.figure(),
#         plt.imshow(image)
        
        image = image.transpose(2,0,1)
            
        id_name = img_name.split("/")[-2]
        id_idx = self.identities.index(id_name)

        return image,id_idx

In [8]:
class VGG16convnet(nn.Module):
            def __init__(self):
                super(VGG16convnet, self).__init__()
                self.model = models.vgg16_bn(pretrained=True).cuda().float()
                self.model.classifier = nn.Sequential(*list(self.model.classifier.children())[:4])
                
            def forward(self, x):
                return self.model.forward(x)

In [9]:
def train():
    net = VGG16convnet().cuda()   
    data_folder = '/media/jmukherjee/2TB_Store/DNN_Projects/Adversarial/data/VGGFACE2_100/Train/'
    data_source , Filelist, identity_list = create_filelists(data_folder)
    VGGTest = VGGFace2_100_PK(Filelist,identity_list)
    pksampler = PKSampler(data_source, 4 , 6)
    dataloader = DataLoader(VGGTest, batch_sampler=pksampler,
                        shuffle=False, num_workers=4)

    margin = 1.
    loss_fn = OnlineTripletLoss(margin, RandomNegativeTripletSelector(margin))
    lr = 1e-3
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
    n_epochs = 20
    log_interval = 100
    
    for epoch_no in range(n_epochs):
        print("Epoch: "+str(epoch_no))
        epoch_loss = 0.0
        net.train()
        for i, (images, ids) in enumerate(dataloader):
            optimizer.zero_grad()
                      
            embeddings = net(images.float().cuda())
            loss,n_triplets = loss_fn(embeddings, ids.cuda())
            
            loss.backward()
            optimizer.step()
    
            epoch_loss = epoch_loss + loss.item()

            if (i+1)%log_interval == 0:
                print("batch = "+str(i)+" Loss = "+str(loss.item())+ " timestamp = "+str(time.time()))
                gc.collect()
            del embeddings, loss, n_triplets
        print("Epoch Loss = "+str(epoch_loss/i))
            

In [10]:
train()

Epoch: 0


/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be ch

batch = 99 Loss = 1.20749950409 timestamp = 1556654515.19
batch = 199 Loss = 0.973337948322 timestamp = 1556654565.54
batch = 299 Loss = 1.12557399273 timestamp = 1556654616.05
batch = 399 Loss = 1.12340688705 timestamp = 1556654666.43
batch = 499 Loss = 1.05665636063 timestamp = 1556654716.55
batch = 599 Loss = 0.979082405567 timestamp = 1556654766.91
batch = 699 Loss = 0.985109269619 timestamp = 1556654817.13
batch = 799 Loss = 0.953769147396 timestamp = 1556654866.98
batch = 899 Loss = 1.15496349335 timestamp = 1556654916.88
batch = 999 Loss = 1.053642869 timestamp = 1556654966.9
batch = 1099 Loss = 1.05140328407 timestamp = 1556655016.95
Epoch Loss = 3.7218849149
Epoch: 1


/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be ch

batch = 99 Loss = 0.94804418087 timestamp = 1556655068.71
batch = 199 Loss = 1.04183650017 timestamp = 1556655118.87
batch = 299 Loss = 0.988510847092 timestamp = 1556655168.83
batch = 399 Loss = 1.06649029255 timestamp = 1556655218.54
batch = 499 Loss = 0.993713498116 timestamp = 1556655268.25
batch = 599 Loss = 1.03923273087 timestamp = 1556655317.93
batch = 699 Loss = 1.08206963539 timestamp = 1556655367.56
batch = 799 Loss = 1.160551548 timestamp = 1556655417.25
batch = 899 Loss = 0.968509316444 timestamp = 1556655467.25
batch = 999 Loss = 1.03682589531 timestamp = 1556655517.7
batch = 1099 Loss = 1.03084254265 timestamp = 1556655568.07
Epoch Loss = 1.05253632192
Epoch: 2


...

batch = 99 Loss = 0.72160500288 timestamp = 1556663931.05
batch = 199 Loss = 1.01025414467 timestamp = 1556663979.58
batch = 299 Loss = 0.963723003864 timestamp = 1556664028.09
batch = 399 Loss = 1.0181812048 timestamp = 1556664076.57
batch = 499 Loss = 1.06199204922 timestamp = 1556664125.05
batch = 599 Loss = 1.42761564255 timestamp = 1556664173.49
batch = 699 Loss = 1.39896953106 timestamp = 1556664221.95
batch = 799 Loss = 1.60324633121 timestamp = 1556664270.4
batch = 899 Loss = 1.2634704113 timestamp = 1556664318.86
batch = 999 Loss = 1.04856491089 timestamp = 1556664367.31
batch = 1099 Loss = 0.966193914413 timestamp = 1556664415.76
Epoch Loss = 1.059370076
Epoch: 18


/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be ch

batch = 99 Loss = 1.18004119396 timestamp = 1556664465.67
batch = 199 Loss = 0.779054582119 timestamp = 1556664514.12
batch = 299 Loss = 0.952364861965 timestamp = 1556664562.5
batch = 399 Loss = 1.15458238125 timestamp = 1556664610.97
batch = 499 Loss = 1.11334347725 timestamp = 1556664659.38
batch = 599 Loss = 1.90014600754 timestamp = 1556664707.81
batch = 699 Loss = 1.01594626904 timestamp = 1556664756.2
batch = 799 Loss = 1.26804149151 timestamp = 1556664804.6
batch = 899 Loss = 0.870068788528 timestamp = 1556664853.01
batch = 999 Loss = 1.04660260677 timestamp = 1556664901.41
batch = 1099 Loss = 1.25595569611 timestamp = 1556664949.74
Epoch Loss = 1.07885334315
Epoch: 19


/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jmukherjee/anaconda3/envs/dl/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be ch

batch = 99 Loss = 1.26292705536 timestamp = 1556664999.55
batch = 199 Loss = 1.52619576454 timestamp = 1556665047.88
batch = 299 Loss = 1.00296807289 timestamp = 1556665096.22
batch = 399 Loss = 1.17714774609 timestamp = 1556665144.56
batch = 499 Loss = 0.50256562233 timestamp = 1556665192.93
batch = 599 Loss = 1.85220789909 timestamp = 1556665241.25
batch = 699 Loss = 1.20801913738 timestamp = 1556665289.57
batch = 799 Loss = 1.00241720676 timestamp = 1556665337.92
batch = 899 Loss = 1.24110329151 timestamp = 1556665386.25
batch = 999 Loss = 0.877564311028 timestamp = 1556665434.6
batch = 1099 Loss = 1.22260701656 timestamp = 1556665482.91
Epoch Loss = 1.05674749058
